# Introduction to Traces

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

Weave is a toolkit for developing AI-powered applications.

Use Weave traces to capture the inputs, outputs, and internal structure of your Python function automatically to observe and debug LLM applications.

When you decorate a function with `@weave.op`, Weave records a rich trace of the function while it runs, including any nested operations or external API calls. Use the trace to to debug, understand, and visualize interactions between your code and LLM models, without leaving your notebook.

To get started, complete the prerequisites. Then, define a function decorated with `@weave.op` decorator and run it on an example input to track LLM calls. Weave captures and visualizes the trace automatically.

In [8]:
# Ensure your dependencies are installed with:
!pip install --quiet jedi openai weave

In [21]:
import os
import getpass

#@title Set up your credentials
inference_provider = "OpenRouter" #@param ["W&B Inference", "OpenAI", "OpenRouter"]

# Set up your W&B project and credentials
os.environ["WANDB_ENTITY_PROJECT"] = input("Set up your W&B project (team name/project name): ")
os.environ["WANDB_API_KEY"] = getpass.getpass("Set up your W&B API key (Create an API key at https://wandb.ai/settings): ")

# Set up your OpenAI API key
if inference_provider == "OpenAI":
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key (Find it at https://platform.openai.com/api-keys): ")
elif inference_provider == "OpenRouter":
  os.environ["OPENROUTER_API_KEY"] = getpass.getpass("Enter your OpenRouter API Key (Find it at https://openrouter.ai/keys): ")


Set up your W&B project (team name/project name): test6
Set up your W&B API key (Create an API key at https://wandb.ai/settings): ··········
Enter your OpenRouter API Key (Find it at https://openrouter.ai/keys): ··········


In [23]:
from openai import OpenAI
import weave

import os

# The userdata.get() calls are removed as they are redundant when keys are set via os.environ
# and can cause errors if secrets are not defined in Colab's secrets manager.

weave.init(os.environ["WANDB_ENTITY_PROJECT"])

@weave.op  # Decorator to track requests
def create_completion(message: str) -> str:
    if inference_provider == "W&B Inference":
      client = OpenAI(
          base_url="https://api.inference.wandb.ai/v1",
          api_key=os.environ["WANDB_API_KEY"],
          project=os.environ["WANDB_ENTITY_PROJECT"],
      )
      model_name: str = "OpenPipe/Qwen3-14B-Instruct"
    elif inference_provider == "OpenAI":
      client = OpenAI()
      model_name: str = "gpt-4.1-nano"
    elif inference_provider == "OpenRouter":
      client = OpenAI(
          base_url="https://openrouter.ai/api/v1",
          api_key=os.environ["OPENROUTER_API_KEY"],
      )
      # You can choose a model available on OpenRouter, e.g., 'mistralai/mistral-7b-instruct'
      # Check OpenRouter's website for available models.
      model_name: str = "mistralai/mistral-7b-instruct"
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": message},
        ],
    )
    return response.choices[0].message.content


message = "Tell me a joke."
create_completion(message)


Output()

weave: wandb version 0.24.0 has been recalled!  Please upgrade.
weave: Logged in as Weights & Biases user: atropos.
weave: View Weave data at https://wandb.ai/aegis-eternis/test6/weave
weave: 🍩 https://wandb.ai/aegis-eternis/test6/r/call/019c2529-71ad-7d78-9cdb-19c53c2fc4ab


'Here’s a fresh, groan-worthy dad joke for you—guaranteed to bring a smile (or at least a chuckle)!\n\n**"Why did the scarecrow win an award?**\nBecause he was *outstanding* in his field."\n[rd.com](https://www.rd.com/list/funniest-jokes-of-all-time/) (via *Reader’s Digest*)\n\nOr if you prefer something even sillier, here’s another from the same list:\n**"What do you call a bear with no teeth?**\n*A gummy bear*!" [rd.com](https://www.rd.com/list/funniest-jokes-of-all-time/)\n\nWant more? Here are a few quick picks from the sources:\n- **"Why is Peter Pan always flying?**\nBecause he *Neverlands*." [cosmopolitan.com](https://www.cosmopolitan.com/uk/lifestyle/a45035485/funny-jokes/)\n- **"What kind of bagels do pilots eat?**\n*Plain*." [today.com](https://www.today.com/life/inspiration/funny-jokes-rcna138371)\n- **"How do you make holy water?**\nYou boil the *hell* out of it." [tellmeajoke.today](https://tellmeajoke.today/)\n\nFor a **knock-knock joke**, try this classic:\n**"Knock knoc

### Verify API Keys Loaded
Let's check if the API keys are correctly set in the environment variables.

In [6]:
import os

wandb_api_key = os.environ.get("WANDB_API_KEY")
if wandb_api_key:
    print(f"WANDB_API_KEY loaded: {wandb_api_key[:5]}...{wandb_api_key[-5:]}")
else:
    print("WANDB_API_KEY not found in environment variables.")


WANDB_API_KEY loaded: wandb...jV7Wj


In [15]:
import os

openai_api_key = os.environ.get("OPENROUTER_API_KEY")
if openai_api_key:
    print(f"OPENAI_API_KEY loaded: {openai_api_key[:5]}...{openai_api_key[-5:]}")
else:
    print("OPENAI_API_KEY not found in environment variables (this is expected if 'inference_provider' is 'W&B Inference').")


OPENAI_API_KEY not found in environment variables (this is expected if 'inference_provider' is 'W&B Inference').
